In [ ]:
# Say hello to Jupyter
print('Hello Jupyter!')

In [ ]:
# Load environment variables
import os
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv('GROQ_API_KEY')
print(api_key)


In [ ]:
# First interaction with Llama using Groq Serverless API
import os
from dotenv import load_dotenv
from groq import Groq

load_dotenv()
api_key = os.getenv('GROQ_API_KEY')
client = Groq(api_key=api_key)

message = "Hello, what is your name?"

response = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[{"role": "user", "content": message}]
)

print(response.choices[0].message.content)


In [ ]:
import requests
from bs4 import BeautifulSoup

# Static webisite scraper

# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)





In [ ]:
from IPython.display import Markdown, display

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

# print(user_prompt_for(ed))

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

# print(messages_for(ed))

def summarize(url):
    website = Website(url)
    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=messages_for(website)
    )
    return response.choices[0].message.content

# summarize("https://edwarddonner.com")

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

display_summary("https://edwarddonner.com")

In [ ]:
display_summary("https://openai.com")

In [ ]:
# Dynamic webisite scraper

import asyncio
from playwright.async_api import async_playwright
import nest_asyncio
from bs4 import BeautifulSoup
import time

nest_asyncio.apply()

class Website:
    title: str
    text: str
    url: str

    def __init__(self, url):
        self.url = url
         
    async def run(self, playwright):
        browser = await playwright.chromium.launch(headless=False)
        page = await browser.new_page()
        await page.goto(self.url)
        await page.wait_for_load_state('load')
        
        # Extract data from the page
        self.title = await page.title()
        text = await page.content()
        await browser.close()
    
        soup = BeautifulSoup(text, 'html.parser')
        for irrelevant in soup(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.get_text(separator="\n", strip=True)
    
    async def main(self):
        async with async_playwright() as playwright:
            await self.run(playwright)   

if __name__ == "__main__":
    site = Website('https://openai.com')
    asyncio.run(site.main())
    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=messages_for(site)
    )

    web_summary = response.choices[0].message.content
    display(Markdown(web_summary))